In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [2]:
def curve_fit_plots(day, detector_name, echan, data_type = 'ctime', plot = 'yes', write = 'no'):
    """This function calculates the chi-square fit to the data of a given detector, day and energy channel and saves the figure in the appropriate folder\n
    Input:\n
    calculate.curve_fit_plots( day = YYMMDD, detector (f. e. 'n5'), echan (9 or 129 = total counts for ctime or cspec), data_type = 'ctime' (or 'cspec'), plot = 'yes' (input 'yes!' for creating new plots), write = 'no' (input 'yes' for creating new data_files))\n
    Output:\n
    0 = residuals\n
    1 = counts\n
    2 = fit_curve\n
    3 = cgb_fit\n
    4 = magnetic_fit\n
    5 = earth_ang_bin_fit\n
    6 = sun_ang_bin_fit\n
    7 = crab_ang_bin_fit\n
    8 = plot_time_bin\n
    9 = plot_time_sat"""
    
    det = getattr(detector(), detector_name)
    year = int('20' + str(day)[0:2])
        
    #get the iso-date-format from the day
    date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))
    
    #get the ordinal indicator for the date
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
    
    #read the measurement data
    if data_type == 'ctime':
        ctime_data = readfile.ctime(readfile(), detector_name, day)
        echannels = ctime_data[0]
        total_counts = ctime_data[1]
        echan_counts = ctime_data[2]
        total_rate = ctime_data[3]
        echan_rate = ctime_data[4]
        bin_time = ctime_data[5]
        good_time = ctime_data[6]
        exptime = ctime_data[7]
        bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)
    
        total_rate = np.sum(echan_rate[1:-2], axis = 0)
    
    elif data_type == 'cspec':
        cspec_data = readfile.cspec(readfile(), detector_name, day)
        echannels = cspec_data[0]
        total_counts = cspec_data[1]
        echan_counts = cspec_data[2]
        total_rate = cspec_data[3]
        echan_rate = cspec_data[4]
        bin_time = cspec_data[5]
        good_time = cspec_data[6]
        exptime = cspec_data[7]
        bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)
        
        total_rate = np.sum(echan_rate[1:-2], axis = 0)
        
    else:
        print 'Invalid data type: ' + str(data_type) + '\n Please insert an appropriate data type (ctime or cspec).'
    
    
    if data_type == 'ctime':
        if echan < 9:
            counts = echan_rate[echan]
        elif echan == 9:
            counts = total_rate
            echan = 0
        else:
            print 'Invalid value for the energy channel of this data type (ctime). Please insert an integer between 0 and 9.'
            return
    elif data_type == 'cspec':
        if echan < 129:
            counts = echan_rate[echan]
        elif echan == 129:
            counts = total_rate
            echan = 0
        else:
            print 'Invalid value for the energy channel of this data type (cspec). Please insert an integer between 0 and 129.'
            return
    else:
        print 'Invalid data type: ' + str(data_type) + '\n Please insert an appropriate data type (ctime or cspec).'
        return
                
    #read the satellite data
    sat_data = readfile.poshist_bin(readfile(), day, bin_time_mid, detector_name, data_type)
    sat_time_bin = sat_data[0]
    sat_pos_bin = sat_data[1]
    sat_lat_bin = sat_data[2]
    sat_lon_bin = sat_data[3]
    sat_q_bin = sat_data[4]
        
    #calculate the sun data
    sun_data = calculate.sun_ang_bin(calculate(), detector_name, day, bin_time_mid, data_type)
    sun_ang_bin = sun_data[0]
    sun_ang_bin = calculate.ang_eff(calculate(), sun_ang_bin, echan)[0]
    sun_rad = sun_data[2]
    sun_ra = sun_rad[:,0]
    sun_dec = sun_rad[:,1]
    sun_occ = calculate.src_occultation_bin(calculate(), day, sun_ra, sun_dec, bin_time_mid)[0]
        
    #calculate the crab nebula data
    crab_ra = 83.6330833
    crab_dec = 22.0145
    crab_data = calculate.burst_ang_bin(calculate(), detector_name, day, crab_ra, crab_dec, bin_time_mid, data_type)
    crab_ang_bin = crab_data[0]
    crab_ang_bin = calculate.ang_eff(calculate(), crab_ang_bin, echan, data_type)[0]
    crab_occ = calculate.src_occultation_bin(calculate(), day, crab_ra, crab_dec, bin_time_mid, detector_name, data_type)[0]
        
    #calculate the scox-1 data
    scox_ra = 244.9794583
    scox_dec = -15.64022222
    scox_data = calculate.burst_ang_bin(calculate(), detector_name, day, scox_ra, scox_dec, bin_time_mid, data_type)
    scox_ang_bin = scox_data[0]
    scox_ang_bin = calculate.ang_eff(calculate(), scox_ang_bin, echan, data_type)[0]
    scox_occ = calculate.src_occultation_bin(calculate(), day, scox_ra, scox_dec, bin_time_mid, detector_name, data_type)[0]
        
    #calculate the cygx-1 data
    cygx_ra = 299.5903165
    cygx_dec = 35.20160508
    cygx_data = calculate.burst_ang_bin(calculate(), detector_name, day, cygx_ra, cygx_dec, bin_time_mid, data_type)
    cygx_ang_bin = cygx_data[0]
    cygx_ang_bin = calculate.ang_eff(calculate(), cygx_ang_bin, echan, data_type)[0]
    cygx_occ = calculate.src_occultation_bin(calculate(), day, cygx_ra, cygx_dec, bin_time_mid, detector_name, data_type)[0]
        
    #calculate the earth data
    earth_data = calculate.earth_ang_bin(calculate(), detector_name, day, bin_time_mid, data_type)
    earth_ang_bin = earth_data[0]
    #earth_ang_bin = calc.ang_eff(earth_ang_bin, echan)[0]
    earth_ang_bin = calculate.earth_occ_eff(calculate(), earth_ang_bin, echan)
        
    #read the SFL data
    flares = readfile.flares(readfile(), year)
    flares_day = flares[0]
    flares_time = flares[1]
    if np.any(flares_day == day) == True:
        flares_today = flares_time[:,np.where(flares_day == day)]
        flares_today = np.squeeze(flares_today, axis=(1,))/3600.
    else:
        flares_today = np.array(-5)
        
    #read the mcilwain parameter data
    sat_time = readfile.poshist(readfile(), day)[0]
    lat_data = readfile.mcilwain(readfile(), day)
    mc_b = lat_data[1]
    mc_l = lat_data[2]
        
    mc_b = calculate.intpol(calculate(), mc_b, day, 0, sat_time, bin_time_mid)[0]
    mc_l = calculate.intpol(calculate(), mc_l, day, 0, sat_time, bin_time_mid)[0]
        
    magnetic = mc_l
    magnetic = magnetic - np.mean(magnetic, dtype=np.float64)
      
    #constant function corresponding to the diffuse y-ray background
    cgb = np.ones(len(total_rate))
        
    
     
    
       
        
        
    addition = 2000 #time-binning = 0.256 s
    addition_0 = 0
        
        
    saa_exits = [0]
    for i in range(1, len(total_rate)):
        if np.logical_and(total_rate[i-1] == 0, total_rate[i] != 0):
            #print i
            saa_exits.append(i)
    saa_exits = np.array(saa_exits)
    
    if saa_exits[1] - saa_exits[0] < 10:
        addition_0 = addition
        saa_exits = np.delete(saa_exits, 0)
        
        
        
        
    counts_grey = counts
    #cut out the SAA_exit part
    for i in range(0, len(saa_exits)):
        if i == 0 and addition_0 != 0:
            total_rate[saa_exits[i]:saa_exits[i] + addition_0] = 0
        elif i > 0:
            total_rate[saa_exits[i]:saa_exits[i] + addition] = 0
    
    
        
        
    #counts[120000:] = 0
    cgb[np.where(total_rate == 0)] = 0
    earth_ang_bin[np.where(total_rate == 0)] = 0
    sun_ang_bin[np.where(sun_occ == 0)] = 0
    sun_ang_bin[np.where(total_rate == 0)] = 0
    crab_ang_bin[np.where(crab_occ == 0)] = 0
    crab_ang_bin[np.where(total_rate == 0)] = 0
    scox_ang_bin[np.where(scox_occ == 0)] = 0
    scox_ang_bin[np.where(total_rate == 0)] = 0
    cygx_ang_bin[np.where(cygx_occ == 0)] = 0
    cygx_ang_bin[np.where(total_rate == 0)] = 0
    magnetic[np.where(total_rate == 0)] = 0
    counts[np.where(total_rate == 0)] = 0
     
    #remove vertical movement from scaling sun_ang_bin and crab_ang_bin
    sun_ang_bin[sun_ang_bin>0] = sun_ang_bin[sun_ang_bin>0] - np.min(sun_ang_bin[sun_ang_bin>0])
    crab_ang_bin[crab_ang_bin>0] = crab_ang_bin[crab_ang_bin>0] - np.min(crab_ang_bin[crab_ang_bin>0])
    scox_ang_bin[scox_ang_bin>0] = scox_ang_bin[scox_ang_bin>0] - np.min(scox_ang_bin[scox_ang_bin>0])
    cygx_ang_bin[cygx_ang_bin>0] = cygx_ang_bin[cygx_ang_bin>0] - np.min(cygx_ang_bin[cygx_ang_bin>0])    
        
        
        
        
        
    def exp_func(x, a, b, i, addition):
    #    if saa_exits[i] == 0:
    #        addition = 0
    #    elif saa_exits[i] < 200:
    #        addition = 10
    #    else:
    #        addition = 9
    #    for i in range(0, 30):
    #        if i > addition:
    #            addition = i
    #            break
    #    print addition
    #    addition = round(addition)
        x_func = x[saa_exits[i]+math.fabs(addition):] - x[saa_exits[i]+math.fabs(addition)]
        func = math.fabs(a)*np.exp(-math.fabs(b)*x_func)
        zeros = np.zeros(len(x))
        zeros[saa_exits[i]+math.fabs(addition):] = func
        zeros[np.where(total_rate==0)] = 0
        return zeros
        
        
        
        
        
        
    deaths = len(saa_exits)
      
    exp = []
    for i in range(0, deaths):
        exp = np.append(exp, [4., 0.01])
        
        
        
        
        
        
        
    def fit_function(x, a, b, c, d, e, f, g, addition, addition_0, exp1, exp2, deaths):
        this_is_it = a*cgb + b*magnetic + c*earth_ang_bin + d*sun_ang_bin + e*crab_ang_bin + f*scox_ang_bin + g*cygx_ang_bin
        for i in range(0, deaths):
            if i == 0:
                this_is_it = np.add(this_is_it, exp_func(x, exp1[i], exp2[i], i, addition_0))
            else:
                this_is_it = np.add(this_is_it, exp_func(x, exp1[i], exp2[i], i, addition))
        return this_is_it
     
    def wrapper_fit_function(x, deaths, addition, addition_0, a, b, c, d, e, f, g, *args):
        exp1 = args[::2]
        exp2 = args[1::2]
        return fit_function(x, a, b, c, d, e, f, g, addition, addition_0, np.fabs(exp1), np.fabs(exp2), deaths)


        


        
        
    x0 = np.append(np.array([1300., 20., -12., -1., -1., -1., -1., addition]), exp)
    sigma = np.array((counts + 1)**(0.5))
     
    fit_results = optimization.curve_fit(lambda x, a, b, c, d, e, f, g, *args: wrapper_fit_function(x, deaths, addition, addition_0, a, b, c, d, e, f, g, *args), bin_time_mid, counts, x0, sigma, maxfev = 10000)
    coeff = fit_results[0]
    #pcov = fit_results[1]
     
    #print 'pcov: ', pcov
    #print 'CGB coefficient:',coeff[0]
    #print 'Magnetic field coefficient:',coeff[1]
    #print 'Earth angle coefficient:',coeff[2]
    #print 'Sun angle coefficient:',coeff[3]
    #print 'Addition: ', coeff[4]
    #print 'starting exp:',coeff[5],'&',coeff[6]
    #print 'first SAA:',coeff[7],'&',coeff[8]
        
        
        
        
        
        
    a = coeff[0]
    b = coeff[1]
    c = coeff[2]
    d = coeff[3]
    e = coeff[4]
    f = coeff[5]
    g = coeff[6]
    exp1 = coeff[8::2]
    exp2 = coeff[9::2]
     
    fit_curve = fit_function(bin_time_mid, a, b, c, d, e, f, g, addition, addition_0, exp1, exp2, deaths)
       
        
        
        
        
        
        
        
        
    #####plot-algorhythm#####
    #convert the x-axis into hours of the day
    plot_time_bin_date = calculate.met_to_date(calculate(), bin_time_mid)[0]
    plot_time_bin = (plot_time_bin_date - calculate.day_to_met(calculate(), day)[1])*24#Time of day in hours
    plot_time_sat_date = calculate.met_to_date(calculate(), sat_time_bin)[0]
    plot_time_sat = (plot_time_sat_date - calculate.day_to_met(calculate(), day)[1])*24#Time of day in hours
        
        

    ###plot each on the same axis as converted to counts###
    if plot == 'yes' or plot == 'yes!':
        fig, ax1 = plt.subplots()
            
        plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
        #plot10 = ax1.plot(plot_time_bin, counts_grey, 'g--', label = 'Countrate')
        plot2 = ax1.plot(plot_time_bin, fit_curve, 'r-', label = 'Fit')
        plot3 = ax1.plot(plot_time_sat, d*sun_ang_bin, 'y-', label = 'Sun angle')
        plot4 = ax1.plot(plot_time_sat, c*earth_ang_bin, 'c-', label = 'Earth angle')
        plot5 = ax1.plot(plot_time_sat, b*magnetic, 'g-', label = 'Magnetic field')
        plot6 = ax1.plot(plot_time_sat, a*cgb, 'b--', label = 'Cosmic y-ray background')
        plot7 = ax1.plot(plot_time_sat, e*crab_ang_bin, 'm-', label = 'Crab nebula')
        plot8 = ax1.plot(plot_time_sat, f*scox_ang_bin, 'k-', label = 'Scorpion X-1')
        plot9 = ax1.plot(plot_time_sat, g*cygx_ang_bin, 'm--', label = 'Cygnus X-1')
        #plot8 = ax1.plot(plot_time_sat, geo_orb, 'g--', label = 'Geographical orbit')
            
        #plot vertical lines for the solar flares of the day
        if np.all(flares_today != -5):
            if len(flares_today[0]) > 1:
                for i in range(0, len(flares_today[0])):
                    plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
                    plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
            else:
                plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
                plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
         
        plots = plot1 + plot2 + plot3 + plot4 + plot5 + plot6 + plot7 + plot8 + plot9
        labels = [l.get_label() for l in plots]
        ax1.legend(plots, labels, loc=1)
            
        ax1.grid()
            
        ax1.set_xlabel('Time of day in 24h', fontsize=20)
        ax1.set_ylabel('Countrate', fontsize=20)
        
        #ax1.set_xlim([9.84, 9.85])
        ax1.set_xlim([-0.5, 24.5])
        if counts.all == total_rate.all:
            ax1.set_ylim([-500, 1500])
        else:
            ax1.set_ylim([-500, 750])
            #ax1.set_ylim([-200, 600])
          
        plt.title(data_type + '-countrate-fit of the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)
        
        user = getpass.getuser()
        figure_path = '/home/' + user + '/Work/Fits/' + str(day) + '/'
        if not os.access(figure_path, os.F_OK):
            os.mkdir(figure_path)
        if counts.all == total_rate.all:
            figure_name = 'test_' + str(data_type) + '_' + detector_name + '_tot.png'
        else:
            figure_name = 'test_' + str(data_type) + '_' + detector_name + '_e' + str(echan) + '.png'
        
        fig = plt.gcf() # get current figure
        fig.set_size_inches(20, 12)
        
        figure = os.path.join(figure_path, figure_name)
        plt.savefig(figure, bbox_inches='tight', dpi = 80)
        
        #plt.show()
        
        fig.clf()
        plt.clf()
          
            
            
            
            
        ###plot residual noise of the fitting algorithm###
        residuals = counts - fit_curve
        residuals_grey = counts_grey - fit_curve
        plt.plot(plot_time_bin, residuals_grey, color = 'grey')
        plt.plot(plot_time_bin, residuals, 'b-')
         
        plt.xlabel('Time of day in 24h', fontsize=20)
        plt.ylabel('Residual noise', fontsize=20)
        
        plt.grid()
         
        plt.title(data_type + '-countrate-fit residuals of the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)
         
        plt.xlim([-0.5, 24.5])
        if counts.all == total_rate.all:
            plt.ylim([-600, 600])
        else:
            plt.ylim([-300, 300])
         
        figure_path = '/home/' + user + '/Work/Fits/' + str(day) + '/'
        if not os.access(figure_path, os.F_OK):
            os.mkdir(figure_path)
        if counts.all == total_rate.all:
            figure_name = 'test_' + str(data_type) + '_' + detector_name + '_tot_res.png'
        else:
            figure_name = 'test_' + str(data_type) + '_' + detector_name + '_e' + str(echan) + '_res.png'
        
        fig = plt.gcf() # get current figure
        fig.set_size_inches(20, 12)
        
        figure = os.path.join(figure_path, figure_name)
        plt.savefig(figure, bbox_inches='tight', dpi = 80)
        
        #plt.show()
        
        fig.clf()
        plt.clf()
        


        return residuals, counts, fit_curve, a*cgb, b*magnetic, c*earth_ang_bin, d*sun_ang_bin, e*crab_ang_bin, f*scox_ang_bin, g*cygx_ang_bin, plot_time_bin, plot_time_sat

In [3]:
day = 150926
detector_name = 'n5'
echan = 3


data = curve_fit_plots(day, detector_name, echan, data_type = 'ctime', plot = 'yes', write = 'no')

work_module.py:3294: RuntimeWarning: invalid value encountered in arccos
  ang_det_sun = np.arccos(scalar_product)
work_module.py:1000: RuntimeWarning: invalid value encountered in arccos
  ang_det_burst = np.arccos(scalar_product)
work_module.py:2855: RuntimeWarning: invalid value encountered in arccos
  ang_det_geo = np.arccos(scalar_product)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:225: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:228: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
